In [1]:
%load_ext autotime

import os, math

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm, trange

In [2]:
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Input, BatchNormalization
from sklearn.metrics import mean_squared_error, mean_absolute_error

def model(test_data, train_data):
    test_time = test_data['time']
    test_gl_value = test_data['gl_value']
    
    test_data.drop(columns = ['time'], inplace = True)
    train_data.rename(columns = {'bolus_type_normal dual':'bolus_type_normal_dual', 'bolus_type_square dual':'bolus_type_square_dual'}, inplace = True)
    train_data.drop(columns = ['time'], inplace = True)
    
    empty_train_col = [0]*len(train_data)
    for i, item in enumerate(test_data.columns):
        if item not in train_data.columns:
            train_data.insert(i, item, empty_train_col)

    empty_test_col = [0]*len(test_data)
    for i, item in enumerate(train_data.columns):
        if item not in test_data.columns:
            test_data.insert(i, item, empty_test_col)
    
    X_data = train_data.drop(columns = ['gl_predict'])
    y_data = train_data[['gl_predict']]
    input_dim = X_data.shape[1]
    
    scaler_x = MinMaxScaler()
    scaler_y = MinMaxScaler()
    scaler_x.fit(X_data)
    scaler_y.fit(y_data)
    scaled_X_train_data = scaler_x.transform(X_data)
    scaled_y_train_data = scaler_y.transform(y_data)
    scaled_X_train_data = np.reshape(scaled_X_train_data, (scaled_X_train_data.shape[0], 1, scaled_X_train_data.shape[1]))
    
    model = Sequential()
    model.add(LSTM(128, input_shape = (scaled_X_train_data.shape[1], scaled_X_train_data.shape[2])))
    model.add(Dense(150, activation = 'relu'))
    model.add(Dropout(0.20))
    model.add(Dense(100, activation = 'relu'))
    model.add(Dropout(0.15))
    model.add(Dense(50, activation = 'relu'))
    model.add(Dense(20, activation = 'relu'))
    model.add(Dense(1, activation = 'relu'))
    model.compile(loss = 'mse', optimizer = 'adam')
    model.fit(scaled_X_train_data, scaled_y_train_data, epochs = 200, batch_size = 32, shuffle = False)
    
    X_test_data = test_data.drop(columns = ['gl_predict'])
    y_test_data = test_data[['gl_predict']]
    scaled_X_test_data = scaler_x.transform(X_test_data)
    scaled_X_test_data = np.reshape(scaled_X_test_data, (scaled_X_test_data.shape[0], 1, scaled_X_test_data.shape[1]))
    prediction = model.predict(scaled_X_test_data, batch_size = 32)
    scaled_prediction = scaler_y.inverse_transform(prediction)
    
    final_output = pd.DataFrame(columns = ['timestamp', 'predicted_BGL_value'])
    final_output['timestamp'] = test_time
#     final_output['true_BGL_value'] = test_gl_value
    final_output['predicted_BGL_value'] = scaled_prediction
    
    return (final_output, math.sqrt(mean_squared_error(scaled_prediction, y_test_data)), mean_absolute_error(scaled_prediction, y_test_data))

Using TensorFlow backend.


time: 1.88 s


In [3]:
test_data_list = ['540-ws-training.csv', '544-ws-training.csv', '552-ws-training.csv', '567-ws-training.csv', '584-ws-training.csv', '596-ws-training.csv']

rmse_list = {}
mae_list = {}
for file in os.listdir('../data/derived/5_min_complete_dataset/'):
    if file in test_data_list:
        train_data = pd.read_csv('../data/derived/5_min_complete_dataset/'+file)
        test_data = pd.read_csv('../data/derived_test/5_min_complete_dataset/'+file[0:7]+'testing.csv')
        final_output, rmse_error, mae_error = model(test_data, train_data)
        rmse_list[file] = rmse_error
        mae_list[file] = mae_error
        print ('{} - {}'.format(file, rmse_error))
        print ('{} - {}'.format(file, mae_error))
        final_output.to_csv('../data/output/lstm/'+file[0:7]+'result_5min.csv')

Epoch 1/200
13247/13247 [==============================] - 2s 166us/step - loss: 0.0118
Epoch 2/200
13247/13247 [==============================] - 1s 95us/step - loss: 0.0024
Epoch 3/200
13247/13247 [==============================] - 1s 91us/step - loss: 0.0024
Epoch 4/200
13247/13247 [==============================] - 1s 102us/step - loss: 0.0024
Epoch 5/200
13247/13247 [==============================] - 1s 104us/step - loss: 0.0016
Epoch 6/200
13247/13247 [==============================] - 1s 83us/step - loss: 0.0016
Epoch 7/200
13247/13247 [==============================] - 1s 79us/step - loss: 0.0021
Epoch 8/200
13247/13247 [==============================] - 1s 104us/step - loss: 0.0016
Epoch 9/200
13247/13247 [==============================] - 1s 106us/step - loss: 0.0014
Epoch 10/200
13247/13247 [==============================] - 1s 105us/step - loss: 0.0018
Epoch 11/200
13247/13247 [==============================] - 1s 96us/step - loss: 0.0012
Epoch 12/200
13247/13247 [=========

13247/13247 [==============================] - 3s 239us/step - loss: 6.5364e-04
Epoch 178/200
13247/13247 [==============================] - 3s 255us/step - loss: 6.6978e-04
Epoch 179/200
13247/13247 [==============================] - 3s 246us/step - loss: 6.6319e-04
Epoch 180/200
13247/13247 [==============================] - 3s 247us/step - loss: 7.5532e-04
Epoch 181/200
13247/13247 [==============================] - 3s 228us/step - loss: 6.6892e-04
Epoch 182/200
13247/13247 [==============================] - 3s 227us/step - loss: 6.6136e-04
Epoch 183/200
13247/13247 [==============================] - 3s 225us/step - loss: 6.9021e-04
Epoch 184/200
13247/13247 [==============================] - 3s 233us/step - loss: 7.0114e-04
Epoch 185/200
13247/13247 [==============================] - 3s 221us/step - loss: 6.5324e-04
Epoch 186/200
13247/13247 [==============================] - 3s 228us/step - loss: 6.7449e-04
Epoch 187/200
13247/13247 [==============================] - 3s 238us/step

13535/13535 [==============================] - 2s 178us/step - loss: 4.7497e-04
Epoch 152/200
13535/13535 [==============================] - 2s 169us/step - loss: 4.6801e-04
Epoch 153/200
13535/13535 [==============================] - 2s 166us/step - loss: 5.3068e-04
Epoch 154/200
13535/13535 [==============================] - 2s 171us/step - loss: 4.7950e-04
Epoch 155/200
13535/13535 [==============================] - 2s 167us/step - loss: 5.1152e-04
Epoch 156/200
13535/13535 [==============================] - 2s 164us/step - loss: 4.6607e-04
Epoch 157/200
13535/13535 [==============================] - 2s 156us/step - loss: 5.3255e-04
Epoch 158/200
13535/13535 [==============================] - 2s 176us/step - loss: 5.0567e-04
Epoch 159/200
13535/13535 [==============================] - 2s 173us/step - loss: 5.1599e-04
Epoch 160/200
13535/13535 [==============================] - 2s 175us/step - loss: 4.8337e-04
Epoch 161/200
13535/13535 [==============================] - 2s 162us/step

13628/13628 [==============================] - 2s 150us/step - loss: 3.6359e-04
Epoch 126/200
13628/13628 [==============================] - 2s 148us/step - loss: 3.7274e-04
Epoch 127/200
13628/13628 [==============================] - 2s 140us/step - loss: 3.7752e-04
Epoch 128/200
13628/13628 [==============================] - 2s 147us/step - loss: 3.6038e-04
Epoch 129/200
13628/13628 [==============================] - 2s 145us/step - loss: 3.8189e-04
Epoch 130/200
13628/13628 [==============================] - 2s 148us/step - loss: 3.8435e-04
Epoch 131/200
13628/13628 [==============================] - 2s 136us/step - loss: 3.7370e-04
Epoch 132/200
13628/13628 [==============================] - 2s 156us/step - loss: 3.8315e-04
Epoch 133/200
13628/13628 [==============================] - 2s 154us/step - loss: 3.5960e-04
Epoch 134/200
13628/13628 [==============================] - 2s 144us/step - loss: 3.7427e-04
Epoch 135/200
13628/13628 [==============================] - 2s 136us/step

11096/11096 [==============================] - 2s 141us/step - loss: 0.0015
Epoch 13/200
11096/11096 [==============================] - 2s 144us/step - loss: 0.0011
Epoch 14/200
11096/11096 [==============================] - 2s 137us/step - loss: 0.0013
Epoch 15/200
11096/11096 [==============================] - 2s 146us/step - loss: 0.0013
Epoch 16/200
11096/11096 [==============================] - 2s 142us/step - loss: 8.9002e-04
Epoch 17/200
11096/11096 [==============================] - 2s 139us/step - loss: 9.8480e-04
Epoch 18/200
11096/11096 [==============================] - 2s 136us/step - loss: 8.7165e-04
Epoch 19/200
11096/11096 [==============================] - 2s 141us/step - loss: 8.3950e-04
Epoch 20/200
11096/11096 [==============================] - 2s 143us/step - loss: 9.0332e-04
Epoch 21/200
11096/11096 [==============================] - 2s 145us/step - loss: 9.1070e-04
Epoch 22/200
11096/11096 [==============================] - 2s 149us/step - loss: 8.0850e-04
Epoch 

11096/11096 [==============================] - 2s 144us/step - loss: 5.2136e-04
Epoch 188/200
11096/11096 [==============================] - 2s 145us/step - loss: 4.9982e-04
Epoch 189/200
11096/11096 [==============================] - 2s 144us/step - loss: 5.3849e-04
Epoch 190/200
11096/11096 [==============================] - 2s 152us/step - loss: 5.1954e-04
Epoch 191/200
11096/11096 [==============================] - 2s 149us/step - loss: 5.0962e-04
Epoch 192/200
11096/11096 [==============================] - 2s 151us/step - loss: 5.4561e-04
Epoch 193/200
11096/11096 [==============================] - 2s 141us/step - loss: 5.2683e-04
Epoch 194/200
11096/11096 [==============================] - 2s 138us/step - loss: 5.2295e-04
Epoch 195/200
11096/11096 [==============================] - 2s 145us/step - loss: 5.2163e-04
Epoch 196/200
11096/11096 [==============================] - 2s 143us/step - loss: 5.2319e-04
Epoch 197/200
11096/11096 [==============================] - 2s 143us/step

13104/13104 [==============================] - 2s 128us/step - loss: 2.3779e-04
Epoch 162/200
13104/13104 [==============================] - 2s 121us/step - loss: 2.2886e-04
Epoch 163/200
13104/13104 [==============================] - 2s 129us/step - loss: 2.2584e-04
Epoch 164/200
13104/13104 [==============================] - 2s 126us/step - loss: 2.2270e-04
Epoch 165/200
13104/13104 [==============================] - 2s 123us/step - loss: 2.3549e-04
Epoch 166/200
13104/13104 [==============================] - 2s 121us/step - loss: 2.4961e-04
Epoch 167/200
13104/13104 [==============================] - 2s 128us/step - loss: 2.2151e-04
Epoch 168/200
13104/13104 [==============================] - 2s 119us/step - loss: 2.4082e-04
Epoch 169/200
13104/13104 [==============================] - 2s 117us/step - loss: 2.3420e-04
Epoch 170/200
13104/13104 [==============================] - 2s 119us/step - loss: 2.3889e-04
Epoch 171/200
13104/13104 [==============================] - 2s 120us/step

13104/13104 [==============================] - 2s 117us/step - loss: 4.7188e-04
Epoch 136/200
13104/13104 [==============================] - 2s 116us/step - loss: 4.3527e-04
Epoch 137/200
13104/13104 [==============================] - 2s 121us/step - loss: 4.0900e-04
Epoch 138/200
13104/13104 [==============================] - 2s 129us/step - loss: 4.0774e-04
Epoch 139/200
13104/13104 [==============================] - 2s 127us/step - loss: 3.9987e-04
Epoch 140/200
13104/13104 [==============================] - 2s 127us/step - loss: 4.1424e-04
Epoch 141/200
13104/13104 [==============================] - 2s 117us/step - loss: 4.0428e-04
Epoch 142/200
13104/13104 [==============================] - 2s 136us/step - loss: 4.1183e-04
Epoch 143/200
13104/13104 [==============================] - 2s 130us/step - loss: 4.0020e-04
Epoch 144/200
13104/13104 [==============================] - 2s 120us/step - loss: 4.0492e-04
Epoch 145/200
13104/13104 [==============================] - 2s 118us/step

In [4]:
rmse_list

{'584-ws-training.csv': 17.822750819203726,
 '567-ws-training.csv': 10.678824239511254,
 '596-ws-training.csv': 8.038044629121506,
 '552-ws-training.csv': 7.709657097487749,
 '544-ws-training.csv': 10.744407872246947,
 '540-ws-training.csv': 14.20686363150804}

time: 3.71 ms


In [5]:
mae_list

{'584-ws-training.csv': 15.597565950993785,
 '567-ws-training.csv': 8.194426876161156,
 '596-ws-training.csv': 6.655463285401692,
 '552-ws-training.csv': 6.4766443590177225,
 '544-ws-training.csv': 7.787645732386831,
 '540-ws-training.csv': 12.024127675191808}

time: 9.01 ms


In [6]:
from statistics import pstdev, mean

rmse_values = rmse_list.values()
mae_values = mae_list.values()

print ('Mean - RMSE : {}'.format(mean(rmse_values)))
print ('Standard Deviation - RMSE : {}'.format(pstdev(rmse_values)))

print ('Mean - MAE : {}'.format(mean(mae_values)))
print ('Standard Deviation - MAE : {}'.format(pstdev(mae_values)))

Mean - RMSE : 11.533424714846538
Standard Deviation - RMSE : 3.5330503528269297
Mean - MAE : 9.455978979858832
Standard Deviation - MAE : 3.301733148872038
time: 5.51 ms
